Job Analyzer AI

Goal:
A user pastes a job description (or a URL to one), and this app:

-Extracts skills and requirements

-Tells them what they match / lack

-Suggests what to learn

-Generates tailored resume bullets or a cover letter


In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [12]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')

        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input","nav", "footer", "form", "aside"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

   # # Remove unwanted tags
   #      for tag in soup(["script", "style", "img", "nav", "footer", "form", "aside"]):
   #          tag.decompose()
        
   #      # Try to extract from main or article tag, else fallback to body
   #      main_content = soup.find("main") or soup.find("article") or soup.body
   #      if main_content:
   #          self.text = main_content.get_text(separator="\n", strip=True)
   #      else:
   #          self.text = ""
    def get_contents(self):
        return f"Job Page URL:\n{self.url}\nJob Description:\n{self.text}\n\n"

In [11]:
job_site = Website("https://www.linkedin.com/jobs/view/4193069481/?alternateChannel=search&refId=lafWDnnnJyIYHU1x4F2jkQ%3D%3D&trackingId=HkuAt27ZU%2B1wJkSdFDI5eQ%3D%3D")
print(job_site.text)

Skip to main content
Frontend Developer
Voltora Tech
United States
Apply
Security verification
Frontend Developer
Voltora Tech
United States
11 hours ago
Over 200 applicants
See who Voltora Tech has hired for this role
Apply
Security verification
Save
Report this job
Use AI to assess how you fit
Get AI-powered advice on this job and more exclusive features.
Am I a good fit for this job?
Tailor my resume
Sign in to access AI-powered advices
Sign in
Welcome back
By clicking Continue to join or sign in, you agree to LinkedIn’s
User Agreement
,
Privacy Policy
, and
Cookie Policy
.
New to LinkedIn?
Join now
or
New to LinkedIn?
Join now
By clicking Continue to join or sign in, you agree to LinkedIn’s
User Agreement
,
Privacy Policy
, and
Cookie Policy
.
Sign in to evaluate your skills
Sign in
Welcome back
By clicking Continue to join or sign in, you agree to LinkedIn’s
User Agreement
,
Privacy Policy
, and
Cookie Policy
.
New to LinkedIn?
Join now
or
New to LinkedIn?
Join now
By clicking Con

In [ ]:
job_system_prompt = (
    "You are a helpful assistant that analyzes job descriptions for software developers.\n"
    "Given a job description, extract:\n"
    "- A brief summary of the role\n"
    "- Required skills\n"
    "- Preferred (optional) skills\n"
    "- A list of matched and missing skills (based on user skills provided)\n"
    "- Resume bullet point suggestions\n"
    "- A short custom paragraph for a cover letter\n"
    "Respond in JSON format with those keys: summary, required_skills, preferred_skills, matched_skills, missing_skills, resume_bullets, cover_letter."
    
)


In [13]:
print(job_system_prompt)

You are a helpful assistant that analyzes job descriptions for software developers.
Given a job description, extract:
- A brief summary of the role
- Required skills
- Preferred (optional) skills
- A list of matched and missing skills (based on user skills provided)
- Resume bullet point suggestions
- A short custom paragraph for a cover letter
Respond in JSON format with those keys: summary, required_skills, preferred_skills, matched_skills, missing_skills, resume_bullets, cover_letter.


In [14]:
def get_job_user_prompt(job_description: str, user_skills: list[str]) -> str:
    skills_string = ", ".join(user_skills)
    return f"""Here is a job description:

{job_description}

The user's current skills are: {skills_string}

Please analyze the job post, extract the required and preferred skills, and compare with the user’s skills. Then generate a short resume bullet and a custom paragraph for a cover letter. Respond in JSON."""


In [16]:
job_description = """We are looking for a frontend developer with strong experience in React, TypeScript, and Tailwind CSS. Familiarity with REST APIs and CI/CD is a plus."""

user_skills = ["HTML", "CSS", "JavaScript", "Vue.js", "TypeScript"]

print(get_job_user_prompt(job_description,user_skills
))

Here is a job description:

We are looking for a frontend developer with strong experience in React, TypeScript, and Tailwind CSS. Familiarity with REST APIs and CI/CD is a plus.

The user's current skills are: HTML, CSS, JavaScript, Vue.js, TypeScript

Please analyze the job post, extract the required and preferred skills, and compare with the user’s skills. Then generate a short resume bullet and a custom paragraph for a cover letter. Respond in JSON.


In [25]:
def analyze_job(job_description, user_skills):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": job_system_prompt},
            {"role": "user", "content": get_job_user_prompt(job_description, user_skills)}
        ],
        stream=True
    )
    response = ""

    display_handle = display(Markdown(""), display_id=True) 
    
    for chunk in stream:
        content = chunk.choices[0].delta.content or ""
        response += content
        display_handle.update(Markdown(response))  

    return response

In [28]:
job_site = "https://www.linkedin.com/jobs/collections/recommended/?currentJobId=4172253511"
job_description = Website(job_site).text
user_skills = ["JavaScript", "React", "Tailwind", "Git", "TypeScript"]

result = analyze_job(job_description, user_skills)



```json
{
  "summary": "The role involves developing high-quality web applications with a focus on user interface design, utilizing modern frameworks and tools.",
  "required_skills": [
    "JavaScript",
    "React",
    "HTML",
    "CSS",
    "Git",
    "TypeScript"
  ],
  "preferred_skills": [
    "Tailwind CSS",
    "Redux",
    "Node.js",
    "Webpack",
    "Testing frameworks (e.g., Jest, Mocha)"
  ],
  "matched_skills": [
    "JavaScript",
    "React",
    "Git",
    "TypeScript",
    "Tailwind"
  ],
  "missing_skills": [
    "HTML",
    "CSS",
    "Redux",
    "Node.js",
    "Webpack",
    "Testing frameworks"
  ],
  "resume_bullets": [
    "Developed responsive web applications using JavaScript and React, ensuring optimal user experiences.",
    "Collaborated with UI/UX designers to implement designs using Tailwind CSS.",
    "Utilized Git for version control in an Agile environment, managing feature branches and pull requests."
  ],
  "cover_letter": "I am excited to apply for the web developer position. With strong skills in JavaScript, React, and Tailwind CSS, I am passionate about creating user-friendly applications and have a track record of delivering high-quality code. I am eager to bring my expertise in modern web technologies to your team and contribute to innovative projects."
}
```